You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [12]:
import os
from dotenv import load_dotenv
load_dotenv("F:\Social AI\multiagent-framework\.venv\.env")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [13]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [14]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [15]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [16]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [17]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [18]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [19]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [20]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Content Plan Document for Blog Article on Artificial Intelligence**

**1. Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence**

**Trends:**
- Advancements in Generative AI (e.g., ChatGPT, DALL-E).
- Increased focus on AI ethics and regulation.
- The surge in AI-driven automation in various sectors (healthcare, finance, manufacturing).
- AI in Natural Language Processing (NLP) advancements.
- Applications of AI in cybersecurity.

**Key Players:**
- OpenAI
- Google DeepMind
- Microsoft
- IBM Watson
- NVIDIA
- Amazon Web Services (AWS)
- Facebook AI Research (FAIR)



- Display the results of your execution as markdown in the notebook.

In [25]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# The Evolution and Impact of Artificial Intelligence in 2023

Artificial Intelligence (AI) has transformed from a buzzword into a fundamental aspect of modern technology. As we move further into 2023, AI is not just about automation; it embodies a spectrum of capabilities that extend into various sectors—from healthcare to finance and even cybersecurity. This evolution presents immense opportunities while also introducing significant challenges and ethical considerations that compel us to navigate this landscape wisely.

With innovation accelerating at an unprecedented pace, it’s crucial to stay informed about the latest trends, recognize key players, and understand the implications of AI technology in our daily lives and businesses. In this article, we will explore the most notable advancements in AI, the leading companies pushing these innovations, and the future landscape of AI technology.

## Current Trends in AI

### Advancements in Generative AI

One of the standout trends in AI is the remarkable advancements in generative AI, with tools like ChatGPT and DALL-E redefining creative possibilities. These systems allow users to generate text, images, and even sound that mimic human creativity, finding applications ranging from content creation to art generation. Resources such as [Technology Review](https://www.technologyreview.com) highlight how these tools empower artists and marketers to push boundaries and engage audiences in new ways.

Alongside this innovation, the increased focus on AI ethics and regulation signifies a critical turning point. As AI technologies become more integrated into societal structures, concerns regarding algorithmic bias, transparency, and accountability have intensified. Legislative bodies, particularly in the European Union, are beginning to propose regulations designed to ensure AI development is ethical and responsible.

### The Importance of AI Ethics

The shift towards integrating AI ethics within development is not just timely but necessary. Major companies are starting to recognize the importance of ethical frameworks when creating AI systems, as they grapple with pressing issues like data privacy and bias mitigation. The call for clear governance and ethical guidelines in AI is echoed in reports from institutions such as MIT, emphasizing the need for a balanced approach as we advance our innovations in AI technology.

## Impact of AI on Industries

### Healthcare: Revolutionizing Patient Care

The impact of AI on industries is profound, particularly within healthcare. Techniques such as predictive analytics enable more personalized treatments and improved diagnostics, thereby revolutionizing patient care delivery. According to recent reports by McKinsey, AI has the potential to significantly enhance efficiencies and effectiveness in healthcare, ultimately leading to better health outcomes.

As AI continues to evolve, it is essential to monitor changes in regulatory policies surrounding its applications in healthcare. While the benefits are promising, the ethical implications of data usage must never be overlooked. Hospitals and healthcare institutions must remain vigilant about patient confidentiality and ensuring informed consent as they harness the power of AI.

### Finance: Enhancing Security and Efficiency

In the financial sector, AI is paving the way for enhanced security and efficiency through systems like fraud detection and algorithmic trading. Financial institutions are increasingly deploying AI to analyze vast datasets for suspicious activities, working to ensure consumer interests are safeguarded while streamlining operations.

Moreover, AI not only aids in maintaining security and efficiency but also suggests a potential shift in labor dynamics as automation becomes more prevalent. Concerns surrounding job displacement must be addressed proactively with upskilling and reskilling strategies, ensuring that the workforce is adequately prepared for these changes.

## Key Players in AI Technology

### Leading the Charge: Who's Who in AI

Some of the key players at the forefront of AI innovation include OpenAI, Google DeepMind, Microsoft, and IBM Watson. These organizations have made significant strides, pushing the boundaries of what AI can achieve across various domains. Their research and advancements do not only serve their business needs but also elevate industry standards, fostering a competitive landscape.

Additionally, innovations from startups like Hugging Face and Databricks demonstrate that fresh perspectives can significantly influence AI’s evolution. These companies often challenge the status quo, focusing on niche applications and open-source solutions that democratize access to AI technology.

### Noteworthy News in AI

Exciting developments continue to emerge from the AI landscape. Recent collaborations, such as Microsoft's investment in OpenAI, underscore how partnerships can enhance capabilities and accelerate development—providing both parties a competitive edge. Staying informed about noteworthy news is vital for professionals and enthusiasts alike, as it shapes the trajectory of AI growth and application.

## The Future of AI: What Lies Ahead

### The Urgency for Regulatory Frameworks

As we look toward the future of AI, there is an urgency to establish robust regulatory frameworks that safeguard ethical considerations while still fostering innovation. The collaborative nature of AI-human interactions is expected to evolve further, allowing for unprecedented collaboration that amplifies human creativity and productivity.

It is important to note that as we become increasingly reliant on AI, discussions surrounding its ethical implications, job displacement, and accountability will intensify. Striking the right balance between leveraging AI’s potential and maintaining ethical integrity will shape how society adapts to this transformative technology.

### Embracing the Integration of AI

Moving forward, businesses and individuals alike should embrace the potential of AI technology. Integrating AI tools within organizations can enhance efficiency, boost productivity, and drive innovation. As we explore the future of AI technology, fostering a mindset that encourages critical questioning about ethics and societal impact is essential.

## Conclusion: Navigating the AI Landscape

Artificial Intelligence stands at the precipice of a transformative era. Whether you are a tech enthusiast, a business owner, or merely a curious individual, understanding AI’s evolution and its implications can empower you to harness its benefits responsibly. Encourage yourself and your organizations to explore AI tools that can augment your capabilities while remaining informed about ethical practices that will ultimately shape the future of this technology.

In conclusion, the relationship between society and AI must be mutually beneficial; as we move forward, let us do so with a commitment to innovation grounded in ethical integrity. Together, we can navigate this thrilling landscape and ensure that AI serves humanity positively.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Dating in the future"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result.raw)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).